In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-11-23 14:28:04--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  1.89MB/s    in 0.6s    

2024-11-23 14:28:05 (1.89 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [3]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

/var/folders/fh/3_h70c1n4vvf1596hlwxtykw0000gp/T/ipykernel_68677/2750791519.py:3: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  data = torch.tensor(encode(text), dtype=torch.long)


In [8]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]
#block size is not only one sample, it is multiple samples.

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
#this block explains the mulitple samples
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data #decide which training set you are choosing either the val or the train
    ix = torch.randint(len(data) - block_size, (batch_size,)) #this returns a list of 4 integers that are indexes 
    x = torch.stack([data[i:i+block_size] for i in ix]) #a 4 x 8 matrix because you are using the 4 integers above to create 4 randoms sequences and then turning them into a matrix.
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])#same thing as above but for yourn y.
    return x, y

xb, yb = get_batch('train') #REMEMBER XB and YB are both matrices.
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension #range for b is 0-4 
    for t in range(block_size): # time dimension
        context = xb[b, :t+1] 
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__() #initializing the nueral network object from the nn.module from pytorch
        # each token directly reads off the logits for the next token from a lookup table
        #nn.Embedding(vocab_size, vocab_size) is creating a lookup table that is vocab_size x vocab_size dimensions
        #this is initially initialized to random probabilities, each probability represents the logits of the prediction for next character
        #as you train the model the logits will be better tuned to predict the next character/word better
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        #matrix of size vocab_size x vocab_size

    def forward(self, idx, targets=None): #idx is just a 4X8(batch_size vs sequence length) tensor that is being passed in
        # target is another (B x Sequence length tensor that contains the ground truth)
        # idx and targets are both (B,T) tensor of integers
        # after being passed in the to token_embedding_table each token in a sequence is mapped to a logit vector of size vocabulary size
        logits = self.token_embedding_table(idx) # (B,T,C) 
        #takes each of the elements from the B x T matrix 
        #{24, 43, 58,  5, 57,  1, 46, 43}           #{vector of logits of size 65 based on token: 24} 
        #{44, 53, 56,  1, 58, 46, 39, 58}      ->   #{vector of logits of size 65 based on token: 43} x there should be batch size(4) number of these 
        #{52, 58,  1, 58, 46, 39, 58,  1}      ->   #{vector of logits of size 65 based on token: 58}  
        #{52, 58,  1, 58, 46, 39, 58,  1}           #{vector of logits of size 65 based on token: 5}
                                                    #{vector of logits of size 65 based on token: 57}
                                                    #{vector of logits of size 65 based on token: 1}
                                                    #{vector of logits of size 65 based on token: 46}
                                                    #{vector of logits of size 65 based on token: 43}
        if targets is None: #if targets are provided then then this function computes the loss, otherwise its normal inference
            loss = None
        else:
            B, T, C = logits.shape #32 x 65 matrix because there are 8 tokens per sequence and 4 sequences total.
            logits = logits.view(B*T, C) #this basically converts the dimension of the tensor from (B, T, C) to (B * T, C) which in this case is (32, 65)
            targets = targets.view(B*T) #simply taking the input targets and converting it from dimension batch x sequence
            #and converting it to a single array of length 32
            loss = F.cross_entropy(logits, targets)
            #The logits of shape (B*T, C) (e.g., 32x65) and targets of shape (B*T) (e.g., 32) are passed to F.cross_entropy to compute the loss.
        return logits, loss #returns both the logits and loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens): #this loop iterates max_new_tokens times no index is needed.
            #The model predicts the next token based on the current context (idx).
            #The new token is appended to the context.
            # get the predictions
            logits, loss = self(idx) #the current idx is passed through the models forward function, loss is ignored during generation so logits has shape (B=4, T=8, C=65)
            # focus only on the last time step 
            #shape of logits is (B=4, T=8, C=65)
            logits = logits[:, -1, :] # becomes (B, C) #simple slice to get the logits at the last time step
            # apply softmax to get probabilities
            # First : is selecting all sequences in a batch which in this is case is 4
            # Second -1 is taking only the last token of each batch
            # Third : is taking the entire logit vector of 65 
            # Resulting matrix is a 4 x 65 matrix which contains the last sequence of each batch which is 4. 
            probs = F.softmax(logits, dim=-1) # (B, C) #these are unnormalized scores. Softmax normalizes them into probabilities, summing to 1 for each sequence. 
            # pass the (Batch x Vocabulary size) matrix into a softmax function to get each vector of size 65 to sum to 1(converting them to raw probabilities)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx #Returns a tensor of shape (B, T + max_new_tokens) containing the original context and the newly generated tokens.

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(logits)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
tensor([[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
        [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
        [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
        ...,
        [-2.1910, -0.7574,  1.9656,  ..., -0.3580,  0.8585, -0.6161],
        [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
        [-0.6787,  0.8662, -1.6433,  ...,  2.3671, -0.7775, -0.2586]],
       grad_fn=<ViewBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


### Forward Function
The forward function is going to take a Batch x Sequence length matrix. Once the matrix is passed in, each token in all batch number of sequences is going to be mapped to a vector of dimension vocabulary size(65) of logits of the next predicted token. You will be left with a matrix of size (sequence length x batch size, vocab_size). The target matrix which is also Batch x sequence length, will be converted to an array of size batch x sequence length.


### Generate Function
The generate function is going to taken in an idx matrix(size batch x sequence length) and a max new tokens to generate function. You are going to loop max new tokens times through the function, as each loop you are generating 4 new tokens, one for each sequence. In each loop you will pass in idx(batch x sequence length)to your forward pass function(with no targets passed in because we are generating tokens, not testing). After you will get a (B=4, T=8, C=65) matrix which is 4 matrices of size sequence length x vocabulary size. Each row representing logits of the most likely next token based on the current token. After you get your logits from the forward pass, you are going to logits = logits[:, -1, :]
- The first : represents getting all 4 matrices of sequence length x vocabulary size 
- The second -1 represents getting the last index of each of the four matrices(this represents the most recent outputted token)
- The third : represents keeping all 65 logits and not manipulating the size in any way
The final resulting matrix is a 4 x 65 matrix where each row represents the last token of each sequence and each row is of size vocabulary length which in this case is 65. Once we get out 4 x 65 matrix from choosing the last sequence of each batch, we can pass it in to the softmax function. Here everything is converted from raw logits to probabilities. Lastly these probabilities are going to be passed into the torch.multinomial function which where one token for each of the four sequences is chosen. The resulting shape of passing through the sampling function is a 4x1 matrix. Each of the four rows representing the newly generated token for a sequence(assigned to idx_next):


idx_next = [[35],  [12],  [58],  [44]]

idx_next is then concatenated to idx. This is process is then repeated max new tokens times, each loop representing a new token generation for each batch(last sequence of each batch). Once loop is exited idx is once again returned back.

In [13]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [14]:
batch_size = 32
for steps in range(100): # increase number of steps for good results... (This is training so the more times you loop the better your results can get.)

    # sample a batch of data
    xb, yb = get_batch('train') #randomly sampling 32 different sequences in the training data and their offset by which is the ground truth we calculate our loss with.
    # xb = The input data (features) for the model, typically of shape (batch_size, sequence_length).
    # yb = The ground-truth labels (targets) corresponding to xb.
    # evaluate the loss
    logits, loss = m(xb, yb) #calculating the logits
    #forward pass through the nueral network calucating new tokens and the loss
    #The forward pass computes the predictions (logits) and compares them with the true labels (yb) to calculate the loss.
    optimizer.zero_grad(set_to_none=True)
    #Clears the gradients of all model parameters. Gradients are accumulated by default in PyTorch, so this ensures they don’t build up from previous steps.
    loss.backward()
    #Computes the gradients of the loss with respect to the model’s parameters using the chain rule of differentiation.
    optimizer.step()
    #The optimizer adjusts the model parameters using the gradients computed during loss.backward().

print(loss.item())
#Prints the scalar value of the loss after the current step.

4.65630578994751


In [15]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


oTo.JUZ!!zqe!
xBP qbs$Gy'AcOmrLwwt
p$x;Seh-onQbfM?OjKbn'NwUAW -Np3fkz$FVwAUEa-wzWC -wQo-R!v -Mj?,SPiTyZ;o-opr$mOiPJEYD-CfigkzD3p3?zvS;ADz;.y?o,ivCuC'zqHxcVT cHA
rT'Fd,SBMZyOslg!NXeF$sBe,juUzLq?w-wzP-h
ERjjxlgJzPbHxf$ q,q,KCDCU fqBOQT
SV&CW:xSVwZv'DG'NSPypDhKStKzC -$hslxIVzoivnp ,ethA:NCCGoi
tN!ljjP3fwJMwNelgUzzPGJlgihJ!d?q.d
pSPYgCuCJrIFtb
jQXg
pA.P LP,SPJi
DBcuBM:CixjJ$Jzkq,OLf3KLQLMGph$O 3DfiPHnXKuHMlyjxEiyZib3FaHV-oJa!zoc'XSP :CKGUhd?lgCOF$;;DTHZMlvvcmZAm;:iv'MMgO&Ywbc;BLCUd&vZINLIzkuTGZa
D.?


### Mathematical Trick In Self-Attention

##### consider the following example:


In [18]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape
print(x)

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])


In [19]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B): #iterating through each batch
    for t in range(T): #iterating through each time step
        xprev = x[b,:t+1] # (t,C) 
        xbow[b,t] = torch.mean(xprev, 0)


In [23]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [24]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])